In [1]:
import nltk
import random
import pickle
import json
import numpy as np
import warnings

import tensorflow as tf
import tflearn
from nltk.stem.lancaster import LancasterStemmer

warnings.filterwarnings('ignore')

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [4]:
''' reading json file '''
with open('intents.json') as json_file:
    ''' load data '''
    intents_data = json.load(json_file)

In [5]:
intents_data

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.']},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
  {'tag': 'hours',
   'patterns': ['What hours are you open?',
    'What are your hours?',
    'When are you open?'],
   'responses': ["We're open every day 9am-9pm",
    'Our hours are 9am-9pm every day']},
  {'tag': 'location',
   'patterns': ['What is your location?',
    'Where are you located?',
    'What is your address?',
    'Where is your restaurant situated?'],
   'responses': ['We are on the intersection of Lon

In [6]:
words = []
classes = []
documents= []
ignore_words = ['?']

for intent in intents_data['intents']:
    for pattern in intent['patterns']:
        ''' converting pattern into tokens '''
        tokens = nltk.word_tokenize(pattern)
        
        '''add word to the words list'''
        words.extend(tokens)
        
        '''add word(s) to documents'''
        documents.append((tokens, intent['tag']))
        
        '''add tags to our classes list'''
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [7]:
''' text cleaning '''

''' stemmer '''
stemmer = LancasterStemmer()

'''applying stemming '''
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]

''' sorting words and classes '''
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

In [8]:
words[:10]

["'s",
 'acceiv',
 'address',
 'anyon',
 'ar',
 'bye',
 'can',
 'card',
 'cash',
 'cours']

In [9]:
''' creating training data '''
training = []

'''create an empty array for output'''

output_empty = [0] * len(classes)

for doc in documents:
    
    '''initialize bag of words'''
    
    bag = []
    
    '''list of tokenized words for the pattern'''
        
    ''' stemming '''
    
    pattern_words = [stemmer.stem(word.lower()) for word in doc[0]]
    
    '''creating bag of words array'''
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
        
        ''' output is '1' for current tag and '0' for rest of other tags '''
    
        output_row = list(output_empty)
        output_row[classes.index(doc[1])] = 1

        training.append([bag, output_row])
    
''' shuffling '''

random.shuffle(training)

''' converting training data into numpy array '''

training = np.array(training)

train_x = list(training[:, 0])
train_y = list(training[:, 1])


In [10]:
print(train_x[0][:10])
print(train_y[0][:10])
print("x ",len(train_x[0]))
print("y ",len(train_y[0]))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0]
x  57
y  9


In [11]:
''' Neural mwork Model '''
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net) # tflearn.regression: Defines the loss function and optimization method for training the model. In this case, the default settings will use categorical cross-entropy loss and an optimizer like Adam.

''' Defining model and setting up tensorboard'''
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [28]:
''' training '''
model.fit(train_x, train_y, n_epoch=300, batch_size=8, show_metric=True)

Training Step: 72929  | total loss: 0.00000 | time: 0.998s
| Adam | epoch: 330 | loss: 0.00000 - acc: 1.0000 -- iter: 1760/1767
Training Step: 72930  | total loss: 0.00000 | time: 1.002s
| Adam | epoch: 330 | loss: 0.00000 - acc: 1.0000 -- iter: 1767/1767
--


In [29]:
model.save('model.tflearn')
pickle.dump({'words': words, 'classes': classes, 'train_x': train_x, 'train_y': train_y}, open('training_data', 'wb'))

INFO:tensorflow:C:\Users\Ahmed Ali\Netzwerk academy project\Deep Learning projects\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [30]:
'''restoring all the data structures'''
model.load('./model.tflearn')
data = pickle.load(open('training_data', 'rb'))
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

INFO:tensorflow:Restoring parameters from C:\Users\Ahmed Ali\Netzwerk academy project\Deep Learning projects\model.tflearn


In [31]:
def clean_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words):
    sentence_words = clean_sentence(sentence)
    bag = [0]*len(words)  
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s: 
                bag[i] = 1
                
    return np.array(bag)

In [32]:
ERROR_THRESHOLD = 0.25


def classify(sentence):
    ''' prediction '''
    results = model.predict([bow(sentence, words)])[0]
    
    '''filter out predictions below a threshold'''
    results = [[i, r] for i, r in enumerate(results) if r > ERROR_THRESHOLD]
    
    '''sorting'''
    results.sort(key=lambda x: x[1], reverse=True)
    
    ''' return tuple of intent and probability '''
    return [(classes[r[0]], r[1]) for r in results]

def response(sentence):
    results = classify(sentence)
    '''if we have a classification then find the matching intent tag'''
    if results:
        while results:
            for intent in intents_data['intents']:
                
                '''find a tag matching the first result'''
                if intent['tag'] == results[0][0]:
                    
                    '''a random response from the intent'''
                    return random.choice(intent['responses'])

            results.pop(0)

In [33]:
classify('What are you hours of operation?')

[('hours', 1.0)]

In [34]:
print(response('What are your hours of operation?'))

Our hours are 9am-9pm every day


In [35]:
classify('Do you accept credit cards?')
print(response('Do you accept credit cards?'))


We accept VISA, Mastercard and AMEX


In [36]:
print(classify('Where can we locate you?'))
print(response('Where can we locate you?'))


[('location', 0.99999976)]
Our Address is: 1000 Bridge Avenue, London EC3N 4AJ, UK


In [37]:
print(classify('do you provide home delivery?'))
print(response('do you provide home delivery?'))

[('deliveryoption', 1.0)]
Yes, we provide home delivery through UBER Eats and Zomato?


In [38]:
classes

['deliveryoption',
 'goodbye',
 'greeting',
 'hours',
 'location',
 'menu',
 'payments',
 'thanks',
 'todaysmenu']

You can check out the food menu at www.mymenu.com
